In [ ]:
!pip install openai
!pip install langchain
!pip install dotenv

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
gpt_api_key = os.environ['API2D_API_KEY']

In [3]:
from langchain.chat_models import ChatOpenAI

# chatGPT
chat=ChatOpenAI(
    temperature=0.0,
    model_name="gpt-3.5-turbo-0613", 
    max_tokens=1000,
    openai_api_key=gpt_api_key,
    openai_api_base='https://openai.api2d.net/v1'
)

In [4]:
from langchain.prompts import ChatPromptTemplate

# 用户的商品评价
customer_review = """
MacBook Pro特别棒！特别喜欢！m2处理器性能超强，就是价钱有点小贵！电池续航逆天！不发热！还带有黑科技触控栏！这是我送给奶奶的礼物，但是花了好三天才到。
"""

# Prompt 编写
review_template = """\
从以下的文本提取信息:

gift: 是不是为某个人购买的礼物？如果是的回答True，如果不是回答False
delivery_days: 花了几天收到了礼物？输出一个数字，如果没有这个信息，输出-1
price_value: 获取这个物品的价格或者价值，如果有多个，用逗号分隔组成一个python数组

用以下的键值来格式化信息并输出一个JSON:
gift
delivery_days
price_value

文本: {text}
"""


In [5]:
from langchain.prompts import ChatPromptTemplate

# 创建 ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)
print(messages)

# 请求
response = chat(messages)
print(response.content)


[HumanMessage(lc_kwargs={'content': '从以下的文本提取信息:\n\ngift: 是不是为某个人购买的礼物？如果是的回答True，如果不是回答False\ndelivery_days: 花了几天收到了礼物？输出一个数字，如果没有这个信息，输出-1\nprice_value: 获取这个物品的价格或者价值，如果有多个，用逗号分隔组成一个python数组\n\n用以下的键值来格式化信息并输出一个JSON:\ngift\ndelivery_days\nprice_value\n\n文本: \nMacBook Pro特别棒！特别喜欢！m2处理器性能超强，就是价钱有点小贵！电池续航逆天！不发热！还带有黑科技触控栏！这是我送给奶奶的礼物，但是花了好三天才到。\n\n', 'additional_kwargs': {}}, content='从以下的文本提取信息:\n\ngift: 是不是为某个人购买的礼物？如果是的回答True，如果不是回答False\ndelivery_days: 花了几天收到了礼物？输出一个数字，如果没有这个信息，输出-1\nprice_value: 获取这个物品的价格或者价值，如果有多个，用逗号分隔组成一个python数组\n\n用以下的键值来格式化信息并输出一个JSON:\ngift\ndelivery_days\nprice_value\n\n文本: \nMacBook Pro特别棒！特别喜欢！m2处理器性能超强，就是价钱有点小贵！电池续航逆天！不发热！还带有黑科技触控栏！这是我送给奶奶的礼物，但是花了好三天才到。\n\n', additional_kwargs={}, example=False)]
{
  "gift": true,
  "delivery_days": 3,
  "price_value": ["小贵"]
}
